In [1]:
# Importing Libraries
from PyPDF2 import PdfFileReader
import numpy as np
import tabula
import pandas as pd

# This version is to merge all the Spell Bee Words - WOC and SOW . Get the unique words to upload to Sql DB

Reading all the spellbee Words from spreadsheet

In [2]:
SpellBee_2021WoC_df = pd.read_excel('Bee_Docs/Spelling Bee - Database.xlsx', sheet_name='2021_WoC')
SpellBee_2021School_df = pd.read_excel('Bee_Docs/Spelling Bee - Database.xlsx', sheet_name='2021_School')
SpellBee_2020WoC_df = pd.read_excel('Bee_Docs/Spelling Bee - Database.xlsx', sheet_name='2020_WoC')
SpellBee_2020School_df = pd.read_excel('Bee_Docs/Spelling Bee - Database.xlsx', sheet_name='2020_School')

Extracting only Words

In [7]:
SpellBee_2021 =SpellBee_2021WoC_df.loc[:,['Difficulty Level','Words']]


In [9]:
SpellBee_2021 = pd.merge(SpellBee_2021,SpellBee_2021School_df['Words'],how="outer",on=['Words'])
SpellBee_2021.to_csv('Bee_Docs/2021_WoC.csv')

In [5]:
class Spell_Bee:
    def __init__(self,path):
        # Initialization
        self.words_array=[]
        self.level = 'ONE'
        self.dataset = self.Create_DataFrame()
        self.path = path
    
    def Create_DataFrame(self):
        return pd.DataFrame({'Difficulty Level': self.level, 'Words':self.words_array})

    
    
    #Opening File
    def Open_Pdf(self):
        f = open(self.path,'rb') 
        pdf = PdfFileReader(f)
        tot_pg= pdf.getNumPages()
        print('After getnum pages')
        f.close()
        print('total page: ', tot_pg)       
        return tot_pg
    
    def Append_rows(self,pg):
        def Array_handling(df_data):
            sample_list=''
            for item in df_data:
                sample_list= ','.join(item)
            x=np.array(sample_list.split(','))
            y=np.array(df_data[0])
            return np.append(x,y)
        data=tabula.read_pdf(self.path,pages=pg)
        self.words_array=np.append(self.words_array,Array_handling(data))
    
    def Page_Extract(self,page):
        def Lvl_chk(x):
            num_lst =['ONE','TWO', 'THREE']
            column_val=''

            for i in num_lst:

                if i in x:

                    column_val = i
                    return column_val
            return None
        
    
    
        for pg in range(page+1):
            print('Page no:',pg)
            df_new=tabula.read_pdf("Bee_Word_2020.pdf", pages=pg,area=(30,331,70,420))
            x=str(df_new)


            column_val =Lvl_chk(x)

            if column_val and self.level == column_val:
                self.Append_rows(pg)
                
            elif column_val :
  
                df2 = self.Create_DataFrame()
                self.dataset =pd.concat([self.dataset,df2], ignore_index=True, axis=0)

                self.level = column_val
                self.words_array=[]
                self.Append_rows(pg)
        
        df2 = self.Create_DataFrame()
        self.dataset =pd.concat([self.dataset,df2], ignore_index=True, axis=0)
        
            
filename=input('\nInput Filename(PDF):') 
Output_csv = input('\nOutput Filename(CSV ):')
Sb = Spell_Bee(filename)
page = Sb.Open_Pdf()
Sb.Page_Extract(page)
Output=Sb.dataset
dataframe_out = Output.copy()
# handling null values
Output.dropna(axis=0,how='any',inplace=True)
import re
def Orsplit(x):
    match = re.findall('OR\s*\**([a-zA-Z]+)|\**([a-zA-Z]+)',x)
    for item in match:
        if item[0] == '':
            return item[1]
        return item[0]
Output['Words'] = Output['Words'].apply(Orsplit)

Output.to_csv(Output_csv,encoding='utf-8-sig') # writing into csv format file
print('Output:', Output)

    
    
   


Input Filename(PDF):Bee_Word_2020_decrypt.pdf

Output Filename(CSV ):Bee_Word_2020_Output.csv


'pages' argument isn't specified.Will extract only from page 1 by default.


After getnum pages
total page:  28
Page no: 0
Page no: 1
Page no: 2
Page no: 3
Page no: 4
Page no: 5
Page no: 6
Page no: 7
Page no: 8
Page no: 9
Page no: 10
Page no: 11
Page no: 12
Page no: 13
Page no: 14
Page no: 15
Page no: 16
Page no: 17
Page no: 18
Page no: 19
Page no: 20
Page no: 21
Page no: 22
Page no: 23
Page no: 24
Page no: 25
Page no: 26
Page no: 27
Page no: 28
Output:      Difficulty Level        Words
0                 ONE      punting
1                 ONE         seal
2                 ONE          ash
3                 ONE      filters
4                 ONE      beeswax
...               ...          ...
4080            THREE   epideictic
4081            THREE        zazen
4082            THREE        Foley
4083            THREE    wasteweir
4084            THREE  encarnadine

[4058 rows x 2 columns]


In [108]:
Output.dropna(how='any',axis=0,inplace=True)
Output.to_csv('Spell_Bee_fileV1.csv') # writing into csv format file
print('Output:', Output)

Output:      Difficulty Level          Words
0                 ONE            gel
1                 ONE            day
2                 ONE         scorch
3                 ONE           hear
4                 ONE         sizzle
...               ...            ...
4092            THREE    brucellosis
4093            THREE  **incarnadine
4094            THREE    haplography
4095            THREE         aperçu
4096            THREE          terai

[4060 rows x 2 columns]


In [2]:

import PyPDF2
PATH = 'Bee_Word_2020_decrypt.pdf'
with open(PATH, mode='rb') as f:        
    reader = PyPDF2.PdfFileReader(f)
   
    print(f"Number of page: {reader.getNumPages()}")
   

Number of page: 28


In [25]:
Output = dataframe_out.copy()
# handling null values
Output.dropna(axis=0,how='any',inplace=True)
#Output =['OR etouffee', 'bergère', '**kibitzer', 'force majeure', 'coup de grace', 'OR löss']
import re
def Orsplit(x):
    match = re.findall('OR\s*\**([w+\s*\w*]*)|\**([w+\s*\w*]*)',x)
    for item in match:
        print(item)
        if item[0] == '':
            return item[1]
        return item[0]
Output['Words'] = Output['Words'].apply(Orsplit)

Output.to_csv('Output_Bee.csv') # writing into csv format file


('', 'punting')
('', 'seal')
('', 'ash')
('', 'filters')
('', 'beeswax')
('', 'holler')
('', 'crowd')
('', 'twirled')
('', 'flame')
('', 'cone')
('', 'social')
('', 'billboard')
('', 'secret')
('', 'admit')
('', 'grits')
('', 'never')
('', 'behold')
('', 'chicken')
('', 'fumble')
('', 'grown')
('', 'cliff')
('', 'silly')
('', 'mix')
('', 'puppets')
('', 'pond')
('', 'pieces')
('', 'hem')
('', 'Monday')
('', 'drool')
('', 'waited')
('', 'health')
('', 'brass')
('', 'candles')
('', 'grub')
('', 'understand')
('', 'finish')
('', 'pardon')
('', 'onshore')
('', 'brim')
('', 'hook')
('', 'sound')
('', 'cheer')
('', 'shake')
('', 'stunts')
('', 'plot')
('', 'number')
('', 'benches')
('', 'twisty')
('', 'kiddo')
('', 'deny')
('', 'desk')
('', 'cooking')
('', 'shrugged')
('', 'chips')
('', 'proof')
('', 'jolly')
('', 'bobcat')
('', 'amazed')
('', 'still')
('', 'scoop')
('', 'gorp')
('', 'powder')
('', 'polo')
('', 'collect')
('', 'pranks')
('', 'rover')
('', 'town')
('', 'walnut')
('', 'roller'

('', 'galvanize')
('', 'perseverance')
('', 'embryo')
('', 'ineptitude')
('', 'residue')
('', 'trellis')
('', 'agonistic')
('', 'apparatus')
('', 'languorous')
('', 'academese')
('', 'halibut')
('', 'juxtapose')
('', 'rosin')
('', 'myopic')
('', 'stevia')
('', 'registrar')
('', 'spectrometer')
('', 'hydrangea')
('', 'burial')
('', 'ebullience')
('', 'phycology')
('', 'alma mater')
('', 'circadian')
('', 'slalom')
('', 'bulwark')
('', 'clairvoyance')
('', 'malaise')
('', 'comedienne')
('', 'servitude')
('', 'corollary')
('', 'iridescent')
('', 'impetus')
('', 'staid')
('', 'cutaneous')
('', 'traverse')
('', 'étude')
('', 'quince')
('', 'affianced')
('', 'acerbity')
('', 'gnarled')
('', 'tungsten')
('', 'cranium')
('', 'cyclone')
('', 'archaism')
('', 'inimical')
('', 'hydroponic')
('', 'vineyard')
('', 'equivalent')
('', 'usurper')
('', 'commodious')
('', 'kaleidoscope')
('', 'combustible')
('', 'interrogative')
('', 'portentous')
('', 'phraseology')
('', 'austere')
('', 'gluttonous')
(

('Veronique', '')
('', 'kipuka')
('', 'toccata')
('', 'katsura')
('', 'dragée')
('', 'hummock')
('', 'planetesimal')
('', 'noumenon')
('', 'pertinacity')
('', 'diastole')
('', 'choucroute')
('', 'internecine')
('', 'taoiseach')
('', 'chicanery')
('', 'gasthaus')
('', 'mediobrome')
('', 'hsaing')
('', 'duxelles')
('', 'obeisant')
('', 'vilipend')
('', 'transmontane')
('', 'demurrage')
('', 'stevedore')
('', 'mesial')
('', 'whippoorwill')
('', 'vitiate')
('', 'laterigrade')
('', 'tristeza')
('', 'harangue')
('', 'Feldenkrais')
('', 'Ficus')
('', 'spodumene')
('', 'hyssop')
('', 'trigeminal')
('', 'niveau')
('', 'bailiwick')
('', 'agelicism')
('', 'leberwurst')
('', 'naumachia')
('', 'bruja')
('', 'rouille')
('', 'bozzetto')
('', 'subrident')
('', 'daguerreotype')
('', 'focaccia')
('', 'upsilon')
('', 'rescissible')
('', 'coiffure')
('', 'ethylene')
('', 'yakitori')
('', 'pahoehoe')
('', 'Kjeldahl')
('', 'catachresis')
('', 'attaché')
('', 'nidicolous')
('', 'ardoise')
('', 'neophyte')
('

In [13]:
Output =['OR etouffee', 'bergère', '**kibitzer', 'force majeure', 'coup de grace', 'OR löss']
import re
def Orsplit(x):
    matched = re.match('OR\s*\**(\w+(\s*\w*)*)|\**([\w+\s*\w*]*)',x)
    if matched:
        print(matched.groups())

#Output.to_csv(Output_csv) # writing into csv format file
for i in Output:
    Output_new = Orsplit(i)

    

('etouffee', '', None)
(None, None, 'bergère')
(None, None, 'kibitzer')
(None, None, 'force majeure')
(None, None, 'coup de')
('löss', '', None)


In [29]:
#from io import BytesIO
Bee_data  = Output.copy()
#buffer = BytesIO()
#Bee_data = Bee_data.decode("utf8").encode("latin-1")
Bee_data.to_csv('Output_Bee_decrypt.csv',encoding='utf-8-sig')

In [111]:
import pyttsx3
engine = pyttsx3.init() # object creation
""" RATE"""
rate = engine.getProperty('rate')   # getting details of current speaking rate
print (rate)                        #printing current voice rate
engine.setProperty('rate', 125)     # setting up new voice rate
"""VOICE"""
voices = engine.getProperty('voices')       #getting details of current voice
#engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female
"""VOLUME"""
volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1
#printing current volume level
engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1
"""Speak words"""
engine.say("Hello World!")
engine.runAndWait()
engine.stop()

200


In [120]:
def speak_word(df_level, Output):
    df=Output.copy()
    df=df.loc[df['Difficulty Level'] == df_level]
    print(df)
    #randomly select n number of rows out of the total existing row numbers from the dataframe df without replacement.
    n=1
    df_pick=df.take(np.random.permutation(len(df))[:n])
    print('Random Word ',df_pick.iloc[0][1])
    random_word = df_pick.iloc[0][1]
    engine.say(random_word)
    engine.runAndWait()
    engine.stop()
speak_word('ONE', Output)

    Difficulty Level      Words
0                ONE        gel
1                ONE        day
2                ONE     scorch
3                ONE       hear
4                ONE     sizzle
..               ...        ...
834              ONE     kernel
835              ONE    cornily
836              ONE      frock
837              ONE      gruel
838              ONE  teenagers

[831 rows x 2 columns]
Random Word  medallion


In [ ]:
import speech_recognition as sr 
import pyttsx3  
  
# Initialize the recognizer  
r = sr.Recognizer()  
# Loop infinitely for user to 
# speak 
  
while(1):     
      
    # Exception handling to handle 
    # exceptions at the runtime 
    try: 
          
        # use the microphone as source for input. 
        with sr.Microphone() as source2: 
              
            # wait for a second to let the recognizer 
            # adjust the energy threshold based on 
            # the surrounding noise level  
            r.adjust_for_ambient_noise(source2, duration=0.2) 
              
            #listens for the user's input  
            audio2 = r.listen(source2) 
              
            # Using ggogle to recognize audio 
            MyText = r.recognize_google(audio2) 
            MyText = MyText.lower() 
  
            print("Did you say "+MyText) 
            SpeakText(MyText) 
              
    except sr.RequestError as e: 
        print("Could not request results; {0}".format(e)) 
          
    except sr.UnknownValueError: 
        print("unknown error occured")
def Listen_word(df_level, Output, MyText):
    df=Output.copy()
    df = df.loc[df['Difficulty Level'] == df_level]
    if MyText in df['Words']:
        return True
    else:
        return False
if Listen_word('ONE', Output, MyText):
    engine.say('You are Correct')
    engine.runAndWait()
    engine.stop()
else:
    engine.say('You are Wrong Try some other word')
    engine.runAndWait()
    engine.stop()
    


In [ ]:
from random import sample

# given data frame df

# create random index
rindex =  np.array(sample(xrange(len(df)), 10))

# get 10 random rows from df
dfr = df.ix[rindex]
